## Introducción Scraping III



Trabajo de scraping sobre la web de FIFA para obtener información de los jugadores de la selección española de fútbol en el Mundial de 2022

> Procesado de los html descargados



## Extracción a partir de los archivos descargados

Ya tenemos las selecciones en la carpeta **equipos**. Ahora las podemos leer y procesar como hicimos en el notebook anterior.


In [9]:
# Recuperamos las funciones del ejercicio anterior

from bs4 import BeautifulSoup
import re
from pathlib import Path

ruta = Path('equipos')  # ruta de los archivos html que hemos guardado

def get_soup(f):
    '''Devuelve el objeto BeautifulSoup a partir de la ruta de un archivo .html'''

    with open(f) as f:
        html_doc = f.read()
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup

def extraer_jugador(j):
    '''Extae nombre, apellido, imagen y posición de un jugador a partir de un objeto BeautifulSoup'''
    
    nombre = j.select('div[class*="player-badge-card_playerFirstname"] span.d-none span')[0].text
    # Algunos jugadores no tienen apellido
    apellido_ = j.select('div[class*="player-badge-card_playerSurname"] span.d-none span')
    if apellido_:
        apellido = apellido_[0].text
    else:
        apellido = ''

    text_url = j.select('div[class*="player-badge-card_playerImage"] div')[0].attrs['style']
    # Algunas url no contienen la url de la imagen ¿?
    url = re.search(r'url\("(.*)"\)', text_url ).group(1)
    posicion = j.select('div[class*="player-badge-card_playerPosition"] span.d-none')[0].text

    return {'nombre': nombre, 'apellido': apellido, 'imagen': url, 'posicion': posicion}  

def extraer_plantilla(f):
    '''Extrae los jugadores de la plantilla del archivo f'''
    def asigna_equipo(j):
        j['equipo'] = f.stem
        return j

    soup = get_soup(f)
    #jugadores = soup.select('div[class*="player-badge-card_playerBadgeCard"]')
    jugadores = soup.select('div[class*="player-badge-card_badgeCard"]')
    datos = [extraer_jugador(j) for j in jugadores]
    return map(asigna_equipo, datos)

    

In [11]:
# Lista para guardar los jugadores

jugadores = []

In [14]:
# Recorremos los archivos html de la carpeta equipos y extraemos los jugadores
# con la función extraer_plantilla

for f in ruta.glob('*.html'):
    jugadores.extend(extraer_plantilla(f))
    

In [15]:
# Comprobamos cuánto jugadores hemos extraído
len(jugadores)

857

In [16]:
import json

with open('jugadores_mundial.json', 'w') as f:
    json.dump(jugadores, f, indent = 4)